In [ ]:
#@title Settings
use_ngrok = False #@param {type:"boolean"}
ngrok_auth_key = "" #@param {type:"string"}
branch = 'main' #@param ["main", "dev"]
use_flash_attention = True #@param {type:"boolean"}
save_models_to_gdrive = False #@param {type:"boolean"}
save_images_to_gdrive = False #@param {type:"boolean"}


In [ ]:
#@title Extensions
disabled_extensions = []

image_browser = True #@param {type:"boolean"}
interrogator = True #@param {type:"boolean"}
mesh_gen = True #@param {type:"boolean"}
prompt_styles = True #@param {type:"boolean"}
segmentation = True #@param {type:"boolean"}
upscaler = True #@param {type:"boolean"}
training = True #@param {type:"boolean"}

if not image_browser: disabled_extensions.append('kd-image-browser')
if not interrogator: disabled_extensions.append('kd-interrogator')
if not mesh_gen: disabled_extensions.append('kd-mesh-gen')
if not prompt_styles: disabled_extensions.append('kd-prompt-styles')
if not segmentation: disabled_extensions.append('kd-segmentation')
if not upscaler: disabled_extensions.append('kd-upscaler')
if not training: disabled_extensions.append('kd-training')

disabled_extensions = str.join(',', disabled_extensions)

In [ ]:
if save_models_to_gdrive or save_images_to_gdrive:
  from google.colab import drive

  drive.mount('/content/gdrive')

In [ ]:
!python --version

[gpu] = !nvidia-smi --query-gpu=gpu_name --format=csv,noheader
print (gpu)

import torch
print(torch.__version__)

!nvidia-smi

In [ ]:
%cd /content

!git clone -b {branch} https://github.com/seruva19/kubin.git
%cd /content/kubin

!pip install -r requirements.txt
if use_flash_attention:
  !pip install https://github.com/seruva19/flash-attn-wheels/raw/main/torch2.0.0%2Bcu118/flash_attn-1.0.1-cp310-cp310-linux_x86_64.whl

In [ ]:
if use_ngrok: 
  !pip install pyngrok
  from pyngrok import ngrok
  ngrok.set_auth_token(ngrok_auth_key)

  tunnels = ngrok.get_tunnels()
  for tunnel in tunnels:
    ngrok.disconnect(tunnel.public_url)

  print(ngrok.connect(7860, "http"))

In [ ]:
!python src/kubin.py \
  --cache-dir={"/content/gdrive/MyDrive/kubin/models" if save_models_to_gdrive else "/content/kubin/models"} \
  --output-dir={"/content/gdrive/MyDrive/kubin/output" if save_images_to_gdrive else "/content/kubin/output"} \
  --disabled-extensions={disabled_extensions} \
  --share={"none" if use_ngrok else "gradio"} \
  {"--use-flash-attention" if use_flash_attention else ""}